In [72]:
# -*- coding: utf-8 -*-

import pdb
import random
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split
from scipy import stats
from itertools import combinations
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import f_oneway

"""
The function Measure_Patterns has 3 parameters: X_train, y_train, optional
optional will check if the columns selected is categorical (integers and strings) or numerical (float)
if optional is not provided, then the program will assume that the column has integers values, therefore it will be considered categorical
"""



'\nThe function Measure_Patterns has 3 parameters: X_train, y_train, optional\noptional will check if the columns selected is categorical (integers and strings) or numerical (float)\nif optional is not provided, then the program will assume that the column has integers values, therefore it will be considered categorical\n'

In [74]:
# Load dataset 
data = np.loadtxt("/Users/fabianafazio/Documents/GitHub/BP24/Fabiana/Data/uniform_large_d_1.tex")
#data = np.loadtxt("uniform_small_d_1.tex")
#df = pd.read_csv("/Users/fabianafazio/Documents/GitHub/BP24/Fabiana/Data/lung_cancer_data.csv")
#df.head()

# Creating NumPy array
array = np.array(data)
# Converting to Pandas DataFrame
df = pd.DataFrame(array)
# Displaying the table
print(df)

In [53]:
# Converting 25 columns from numerical floats -> categorical integers
for i in range(25):
    
    df.iloc[:,i] = df.iloc[:,i].round() # Rounding
    df.iloc[:,i] = df.iloc[:,i].astype(int) # Integer
    df.iloc[:,i] = df.iloc[:,i].astype('category') # Categories
    
# Turn label into categorical label
df.iloc[:,150] = df.iloc[:,150].astype('category')


In [76]:
# Split dataset into X_train and y_train
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,1:150], df.iloc[:,-1], test_size=0.2, random_state=52)

In [78]:
# Function Measure_Patterns begins here!
def Measure_Patterns(X_train, y_train, optional=None):
    
    # Initialize empty dataframes for numerical and categorical data
    numerical_df = pd.DataFrame()
    categorical_df = pd.DataFrame()
    
    # Check if the data type is provided for columns
    if optional is None:
        print("Optional parameter not provided. Assuming integers values are categorical")
    
        # Splitting X_train into numerical subset 
        print("\nNumerical DataFrame:")
        numerical_df = X_train.select_dtypes(include = ['int', 'int64', 'float', 'float64'])
        print(numerical_df)

        # Splitting X_train into categorical subset 
        print("Categorical DataFrame:")
        categorical_df = X_train.select_dtypes(include=['object', 'category'])
        print(categorical_df)
    
    else:
        # Create empty numerical & categorical data frames
        numerical = []
        numerical_colnames = []
        categorical = []
        categorical_colnames = []
        
        # Check that length of optional = # of columns in X_train
        # Optional is the column type for X_train, so the lengths should be equal
        if len(optional) == len(X_train.columns):
            # For all the values in optional
            for i in range(len(optional)):
                if optional[i] == True:
                    numerical.append(X_train.iloc[:,i])
                    # TO DO: Save SPECIFIC column name in each loop, order matters
                    # numerical_colnames.append()
                else: 
                    categorical.append(X_train.iloc[:,i])
                    # TO DO: Save SPECIFIC column name in each loop, order matters
                    # categorical_colnames.append()
            # Turn transposed arrays into dataframes
            numerical_df = pd.DataFrame(np.transpose(numerical))
            categorical_df = pd.DataFrame(np.transpose(categorical))
            # TO DO: Re-attach the column names to numerical_df & categorical_df 
            
            print("Numerical DF:")
            print(numerical_df)
            print("Categorical Df")
            print(categorical_df)
            
        else:
            print("The length of X_train and optional are different.")

    return numerical_df, categorical_df

# Get numerical and categorical dataframes
#numerical_df, categorical_df = Measure_Patterns(X_train, y_train)
            

In [80]:
##################### Correlation between columns (numerical) Code ############################
# Takes the X_train data to find correlation between NUMERICAL features
def num_corr(X_train_numerical):
    matrix = X_train_numerical.corr(method='pearson')
    print("---------------------------Correlation Matrix------------------------- \n", matrix)
     
#Calls the function so the matrix prints out    
num_corr(numerical_df)

---------------------------Correlation Matrix------------------------- 
           25        26        27        28        29        30        31   \
25   1.000000  0.358685  0.339666  0.420727  0.325975  0.321370  0.312616   
26   0.358685  1.000000  0.357068  0.351886  0.329074  0.287777  0.380258   
27   0.339666  0.357068  1.000000  0.367739  0.389061  0.341500  0.396636   
28   0.420727  0.351886  0.367739  1.000000  0.335209  0.362763  0.407312   
29   0.325975  0.329074  0.389061  0.335209  1.000000  0.360621  0.343141   
..        ...       ...       ...       ...       ...       ...       ...   
145  0.299274  0.287986  0.303644  0.309662  0.296231  0.337183  0.378580   
146  0.329233  0.318679  0.372271  0.342548  0.358008  0.388022  0.348550   
147  0.261596  0.304371  0.366230  0.362147  0.330776  0.275535  0.350481   
148  0.311470  0.325705  0.249223  0.303436  0.347029  0.317083  0.335171   
149  0.331838  0.376243  0.372860  0.329591  0.386683  0.325197  0.342970   

  

In [82]:
####################### Chi-Square (F vs F) Code ################################################
    
print("\n------------------Chi-Squared for Features v. Features-----------------------")
# Finds dependency between all CATEGORICAL features in X_train
def chi_squared_fvf(X_train_categorical):
        
    # Extract variable names
    variable_names = list(X_train_categorical.columns)

    # Initialize matrices to store chi-squared and p-values
    num_variables = len(variable_names)
    chi_squared = np.zeros((num_variables, num_variables))
    p_values = np.zeros((num_variables, num_variables))

    # Compute chi-squared and p-values for each pair of variables
    for i, j in combinations(range(num_variables), 2):
        contingency_table = pd.crosstab(X_train_categorical.iloc[:, i], X_train_categorical.iloc[:, j])
            
        # Compute chi-squared and p-values
        chi2 = stats.chi2_contingency(contingency_table)[0]
        p = stats.chi2_contingency(contingency_table)[1]
            
        # Assign results to chi_squared and p_values matrices
        chi_squared[i, j] = chi2
        chi_squared[j, i] = chi2  # Assign to symmetric position in the matrix
        p_values[i, j] = p
        p_values[j, i] = p  # Assign to symmetric position in the matrix

    # Create a DataFrame with variable names as index and columns
    chi_squared_df = pd.DataFrame(chi_squared, index=variable_names, columns=variable_names)
    p_values_df = pd.DataFrame(p_values, index=variable_names, columns=variable_names)

    # Printing the matrix-like output with variable names
    print("Chi-Squared Values:")
    print(chi_squared_df)
    print("\nP-Values:")
    print(p_values_df)
    
chi_squared_fvf(categorical_df)


------------------Chi-Squared for Features v. Features-----------------------
Chi-Squared Values:
                0          1          2          3          4          5  \
0        0.000000  15.224160  25.490076  34.849944  22.262568  12.436666   
1       15.224160   0.000000  16.636916  26.793910   5.609747  24.156967   
2       25.490076  16.636916   0.000000  24.384730   9.469731  17.746646   
3       34.849944  26.793910  24.384730   0.000000  13.046184   9.840086   
4       22.262568   5.609747   9.469731  13.046184   0.000000  11.386875   
5       12.436666  24.156967  17.746646   9.840086  11.386875   0.000000   
6       17.914775   6.716693  28.809709  14.819042   4.781560   4.801779   
7       17.086793  14.952261   5.102826  20.019675  12.520304  10.815238   
8       19.341124  13.601784  15.378196  27.070757  20.132751  20.061328   
9       29.477945  17.980589   9.469731  38.982638  14.831678  14.819042   
10      12.681208  12.955651  25.488970   0.855007   4.619875  25

In [84]:
##################### Chi-Square (F vs label column) Code ####################################
    
print("\n------------------------Chi-Square (F vs label column)------------------------")
# Finds dependency between all CATEGORICAL features in X_train & the label in y_train
def chi_squared_fvl(X_train_categorical, y_train):
        
    # Combining CATEGORICAL X_train and y_train
    df = X_train_categorical
    df['label'] = y_train

    # Number of features, excluding label
    var_count = len(df.columns)-1

    # Creates an empty array to print values in a table
    results = []

    for i in range(0, var_count):

        # Create contigency table of all features v. label
        crosstab = pd.crosstab(df.iloc[:, i], df.iloc[:,-1])
            
        # Compute chi-squared and p-values
        chi2 = stats.chi2_contingency(crosstab)[0]
        p = stats.chi2_contingency(crosstab)[1]
            
        # Append results to the list
        results.append({
            "Feature": df.columns[i],
            "Chi Squared Statistic": chi2,
            "P-Value": p})

    # Create a DataFrame from the results
    results_df = pd.DataFrame(results)

    # Print the DataFrame
    print("Label:", df.columns.values[-1])
    print(results_df.to_string(index=False))
    
chi_squared_fvl(categorical_df, y_train)


------------------------Chi-Square (F vs label column)------------------------
Label: label
 Feature  Chi Squared Statistic  P-Value
       0                  319.0 0.473672
       1                  319.0 0.473672
       2                  319.0 0.473672
       3                  319.0 0.473672
       4                  319.0 0.473672
       5                  319.0 0.473672
       6                  319.0 0.473672
       7                  319.0 0.473672
       8                  319.0 0.473672
       9                  319.0 0.473672
      10                  319.0 0.473672
      11                  319.0 0.473672
      12                  319.0 0.473672
      13                  319.0 0.473672
      14                  319.0 0.473672
      15                  319.0 0.473672
      16                  319.0 0.473672
      17                  319.0 0.473672
      18                  319.0 0.473672
      19                  319.0 0.473672
      20                  319.0 0.473672
     

In [86]:
############################# ANOVA (Feature vs Feature) #########################################

print("\n------------------ ANOVA (Feature vs Feature) -----------------------")
# Initialize an empty DataFrame to store ANOVA results
anova_results = pd.DataFrame(index=categorical_df.columns, columns=numerical_df.columns)

# Perform ANOVA for each combination of categorical and numerical columns
for cat_col in categorical_df:
    for num_col in numerical_df:
        groups = []
        for category in X_train[cat_col].unique():
            groups.append(X_train[num_col][X_train[cat_col] == category])
        f_statistic, p_value = f_oneway(*groups)
        anova_results.loc[cat_col, num_col] = f_statistic, p_value

# Create a formatted DataFrame for ANOVA results with labeled interpretation
formatted_results = pd.DataFrame(index=categorical_df.columns, columns=numerical_df.columns)
for num_col in numerical_df:
    for cat_col in categorical_df:
        f_statistic, p_value = anova_results.loc[cat_col, num_col]
        if p_value < 0.05:
            significance = "Significant"
        else:
            significance = "Not Significant"
        formatted_results.loc[cat_col, num_col] = f"F = {f_statistic:.2f}, p = {p_value:.4f} ({significance})"

# Display formatted ANOVA results as a table
print(formatted_results)


------------------ ANOVA (Feature vs Feature) -----------------------


KeyError: 0

# ANOVA FEATURE VS LABEL (PENDING) 

In [89]:
############################# ANOVA (Feature vs Label) #############################
print("\n------------------ ANOVA (Feature vs Label) -----------------------")
# Concatenate X_train and y_train into a single DataFrame for ANOVA comparison
data = pd.concat([X_train, y_train], axis=1)

# Automatically detect all columns as features (numerical and categorical)
all_columns = data.columns

# Identify the last column as the target column
target_column = all_columns[-1]

# Initialize an empty DataFrame to store ANOVA results
anova_results = pd.DataFrame(index=[target_column], columns=all_columns)

# Perform ANOVA for each feature column against the target column
for feature_col in all_columns:
    if feature_col == target_column:
        continue  # Skip the target column itself
    groups = [data[feature_col][data[target_column] == value] for value in data[target_column].unique()]
    f_statistic, p_value = f_oneway(*groups)
    anova_results.loc[target_column, feature_col] = f"F = {f_statistic:.2f}, p = {p_value:.4f}"

# Print ANOVA results for each feature with significance indication
print("ANOVA Results:")
for feature_col in anova_results.columns[:-1]:  # Exclude the last column (target column)
    result = anova_results.loc[target_column, feature_col]
    f_statistic, p_value = result.split(', ')
    f_statistic = float(f_statistic.split(' = ')[1])
    p_value = float(p_value.split(' = ')[1])
    
    if p_value < 0.05:
        significance = "Significant"
    else:
        significance = "Not Significant"
    
    print(f"{feature_col}: {result} ({significance})")



------------------ ANOVA (Feature vs Label) -----------------------


AttributeError: 'DataFrame' object has no attribute 'unique'

In [ ]:
########################## Histogram/Graphing ###############################

print("------------------------Histogram/Graphing-----------------------------")


###### These are just here for now so the histogram and bar graph functions work
# Splitting X_train into numerical subset 
numerical_df = X_train.select_dtypes(include = ["float64"])

# Splitting X_train into categorical subset 
categorical_df = X_train.select_dtypes(exclude=['float64'])
        

# Ensure data is 2D
if numerical_df.ndim == 1:
    numerical_df = numerical_df.reshape(-1, 1)  # Reshape 1D array to 2D array with one column

# Number of features (columns) in the dataset
numerical_num_features = numerical_df.shape[1]


# Loop through each numerical feature
for feature_idx in range(numerical_num_features):
    # Extract the current feature data (column)
    feature_df = numerical_df.iloc[:, feature_idx]

    # Compute histogram with 10 bins
    hist, bin_edges = np.histogram(feature_df, bins=10)

    # Print feature number
    print(f"Feature {feature_idx + 1}:")
    
    # Print bin edges
    print("Bin Edges:", bin_edges)

    # Store bin heights in a list
    bin_heights = []
    bin_heights.extend(hist)
    print("Array with bin heights:", bin_heights)

    # Store bin probabilities in a list and normalize
    bin_probs = []
    bin_probs.extend(hist)
    bin_probs = np.array(bin_probs) / sum(bin_heights)
    print("Array with bin probabilities:", bin_probs)

    # Loop through each bin to print range and probabilities
    for i in range(len(hist)):
        bin_range = f"{bin_edges[i]:.2f} to {bin_edges[i+1]:.2f}"  # Bin range
        bin_probability = hist[i] / sum(hist)  # Bin probability
        print(f"Bin {i + 1} ({bin_range}): Height = {hist[i]}, Probability = {bin_probability:.2f}")

    # Separator between features for clarity
    print("\n" + "="*50 + "\n")

# Calculate and store probabilities for each categorical column
print("Proportions for Label for Categorical Columns:")

for column in categorical_df.columns:
    value_counts = categorical_df[column].value_counts(normalize=True).sort_index()
    # print(f"Probabilities for Categorical Column {column}:")
    print(value_counts)
    print()  # Add an empty line for separation   

In [ ]:
############################ KL Divergence ####################################

optional_test = []

for i in range(151):
    optional_test.append(bool(random.getrandbits(1)))

# Call the measure_patterns function
Measure_Patterns(X_train, y_train)